<a href="https://colab.research.google.com/github/dynle/youtube-hate-speech-classification/blob/master/youtube_hate_speech_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Youtube Hate Speech Classification by BERT**

## Get dataset file from Kaggle and edit the structure of columns
https://www.kaggle.com/surekharamireddy/malignant-comment-classification


In [ ]:
import pandas as pd
df = pd.read_csv('./dataset.csv')
df.head()

,isMalignant,comment_text
0,0,Explanation\nWhy the edits made under my usern...
1,0,D'aww! He matches this background colour I'm s...
2,0,"Hey man, I'm really not trying to edit war. It..."
3,0,"""\nMore\nI can't make any real suggestions on ..."
4,0,"You, sir, are my hero. Any chance you remember..."


In [ ]:
df.info

<bound method DataFrame.info of         isMalignant                                       comment_text
0                 0  Explanation\nWhy the edits made under my usern...
1                 0  D'aww! He matches this background colour I'm s...
2                 0  Hey man, I'm really not trying to edit war. It...
3                 0  "\nMore\nI can't make any real suggestions on ...
4                 0  You, sir, are my hero. Any chance you remember...
...             ...                                                ...
159566            0  ":::::And for the second time of asking, when ...
159567            0  You should be ashamed of yourself \n\nThat is ...
159568            0  Spitzer \n\nUmm, theres no actual article for ...
159569            0  And it looks like it was actually you who put ...
159570            0  "\nAnd ... I really don't think you understand...

[159571 rows x 2 columns]>

In data, 1 denotes a malignant comment, and 0 denotes a normal comment

## Delete \n character in each text data

In [ ]:
df['comment_text']=df['comment_text'].str.replace("\n"," ")

## Grouped data based on 0 and 1 at 'isMalignant' column

In [ ]:
grouped = df.groupby(df.isMalignant)

group_0 = grouped.get_group(0)
group_1 = grouped.get_group(1)
group_0.values

array([[0,
        "Explanation Why the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"],
       [0,
        "D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)"],
       [0,
        "Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info."],
       ...,
       [0,
        'Spitzer   Umm, theres no actual article for prostitution ring.  - Crunch Captain.'],
       [0,
        'And it looks like it was actually you who put on the speedy to have the first version deleted now that I look at it.'],
       [0,
        '" And ... I really don\'t think you unders

## Get 10000 data from dataset and save it in txt file

In [ ]:
with open('dataset.txt','w') as f:
  for line in group_0.values[:5000]:
    f.write(str(line[0])+'\t'+line[1]+'\n')
  for line in group_1.values[:5000]:
    f.write(str(line[0])+'\t'+line[1]+'\n')

## Shuffle the dataset and split into train and test dataset

In [ ]:
!shuf dataset.txt -o shuffled.txt
!head -8000 shuffled.txt > train.txt
!tail -2000 shuffled.txt > test.txt

## Install a python library for BERT

In [ ]:
!pip install transformers==4.5.0 fugashi==1.1.0 ipadic==1.0.0 pytorch-lightning==1.2.10

     |████████████████████████████████| 2.1 MB 5.4 MB/s 
     |████████████████████████████████| 486 kB 47.8 MB/s 
     |████████████████████████████████| 13.4 MB 20.1 MB/s 
     |████████████████████████████████| 841 kB 45.0 MB/s 
     |████████████████████████████████| 3.3 MB 49.1 MB/s 
     |████████████████████████████████| 895 kB 32.6 MB/s 
     |████████████████████████████████| 596 kB 29.5 MB/s 
     |████████████████████████████████| 132 kB 48.6 MB/s 
     |████████████████████████████████| 176 kB 49.0 MB/s 
     |████████████████████████████████| 829 kB 46.5 MB/s 
     |████████████████████████████████| 1.1 MB 48.0 MB/s 
     |████████████████████████████████| 192 kB 48.0 MB/s 
     |████████████████████████████████| 271 kB 46.6 MB/s 
     |████████████████████████████████| 160 kB 47.8 MB/s 
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556723 sha256=31fd5b08fe789da1e6aff32d820d1fdc6d53a12d81f05931ac16bb65e70535df
  Stored in directory: /root/.cache

## 学習データを読み込みます．
## Load the train data

In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertForSequenceClassification

# 日本語の事前学習モデル
# MODEL_NAME = 'cl-tohoku/bert-base-japanese-whole-word-masking'
MODEL_NAME = 'bert-base-uncased'

# 学習データの読み込み
train_lines = [x.rstrip().split('\t')[1] for x in open("train.txt").readlines()]
train_labels = [int(x.split('\t')[0]) for x in open("train.txt").readlines()]

# テストデータの読み込み
test_lines = [x.rstrip().split('\t')[1] for x in open("test.txt").readlines()]
test_labels = [int(x.split('\t')[0]) for x in open("test.txt").readlines()]

# 単語分割モデルの読み込み
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

def create_dataset_for_loader(lines, labels):
  dataset_for_loader = []
  for i in range(len(lines)):
    encoding = tokenizer(lines[i],max_length=128,padding='max_length',truncation=True)
    encoding['labels'] = labels[i]
    encoding = { k: torch.tensor(v) for k, v in encoding.items() }
    dataset_for_loader.append(encoding)
  return dataset_for_loader

dataset_for_loader_train = create_dataset_for_loader(train_lines, train_labels)
dataset_for_loader_test = create_dataset_for_loader(test_lines, test_labels)

dataset_train = dataset_for_loader_train[1000:] # 学習データ
dataset_val = dataset_for_loader_train[:1000] # 検証データ
dataset_test = dataset_for_loader_test # 評価データ

dataloader_train = DataLoader(
    dataset_train, batch_size=16, shuffle=True
) 
dataloader_val = DataLoader(dataset_val, batch_size=16)
dataloader_test = DataLoader(dataset_test, batch_size=1)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

## モデルの定義をします．
## Define a model 

In [ ]:
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader

from transformers import BertJapaneseTokenizer, BertForSequenceClassification
import pytorch_lightning as pl

class BertForSequenceClassification_pl(pl.LightningModule):
    def __init__(self, model_name, num_labels, lr):
        super().__init__()
        self.save_hyperparameters() 
        self.bert_sc = BertForSequenceClassification.from_pretrained(
            model_name, num_labels=num_labels)
        self.test_results = []
        
    def training_step(self, batch, batch_idx):
        output = self.bert_sc(**batch)
        loss = output.loss
        self.log('train_loss', loss) 
        return loss
        
    def validation_step(self, batch, batch_idx):
        output = self.bert_sc(**batch)
        val_loss = output.loss
        self.log('val_loss', val_loss)

    def reset_test_results(self):
        self.test_results = []

    def test_step(self, batch, batch_idx):
        labels = batch.pop('labels')
        output = self.bert_sc(**batch)
        probs = torch.nn.functional.softmax(output.logits,dim=-1)
        labels_predicted = output.logits.argmax(-1)
        num_correct = ( labels_predicted == labels ).sum().item()
        accuracy = num_correct/labels.size(0) 
        hyp = labels_predicted.cpu().numpy()[0]
        ref = labels.cpu().numpy()[0]
        prob = probs.cpu().numpy()[0][hyp]
        self.test_results.append({"hyp":hyp, "ref": ref, "prob":prob})
        self.log('accuracy', accuracy)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)

# 学習時にモデルの重みを保存する条件を指定
checkpoint = pl.callbacks.ModelCheckpoint(
    monitor='val_loss',
    mode='min',
    save_top_k=1,
    save_weights_only=True,
    dirpath='model/',
)

# 学習が進まなくなったら終了する条件を指定
early_stopping = pl.callbacks.EarlyStopping(
    min_delta=0.00,
    patience=1,
    verbose=True,
    monitor='val_loss',
    mode='min',
)    

# 学習の方法を指定
trainer = pl.Trainer(
    gpus=1, 
    max_epochs=10,
    callbacks = [checkpoint,early_stopping]
)

# 学習に利用するモデルの作成
model = BertForSequenceClassification_pl(MODEL_NAME, num_labels=2, lr=1e-5)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

## 学習の実行
## Run training

In [ ]:
trainer.fit(model, dataloader_train, dataloader_val) 
best_model_path = checkpoint.best_model_path # ベストモデルのファイル
print('ベストモデルのファイル: ', checkpoint.best_model_path)
print('ベストモデルの検証データに対する損失: ', checkpoint.best_model_score)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type                          | Params
----------------------------------------------------------
0 | bert_sc | BertForSequenceClassification | 109 M 
----------------------------------------------------------
109 M     Trainable params
0         Non-trainable params
109 M     Total params
437.935   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

ベストモデルのファイル:  /content/model/epoch=1-step=875.ckpt
ベストモデルの検証データに対する損失:  tensor(0.2079, device='cuda:0')


## 分類の実行と結果の表示．連番，正解ラベル，予測ラベル，確率，本文の順番で結果が出ます．
## Run a classification and show a result

In [ ]:
model.reset_test_results()
test = trainer.test(test_dataloaders=dataloader_test)

for i in range(len(test_lines)):
  line = test_lines[i]
  label = test_labels[i]
  d = model.test_results[i]
  hyp = d['hyp'].item()
  prob = d['prob'].item() 
  # print(f"{i+1}\t{label}\t{hyp}\t{prob}\t{line}")

print(f'Accuracy: {test[0]["accuracy"]:.3f}')

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'accuracy': 0.9325000047683716}
--------------------------------------------------------------------------------
Accuracy: 0.933


### Classify youtube comments extracted by Youtube API
#### => Check whether each comment is a hate speech or not

In [ ]:
comment_lines = [x.strip() for x in open("comments.txt").readlines()]
comment_labels = [1 for x in comment_lines]

dataset_for_loader_comment = create_dataset_for_loader(comment_lines, comment_labels)
dataloader_comment = DataLoader(dataset_for_loader_comment, batch_size=1)

model.reset_test_results()
test = trainer.test(test_dataloaders=dataloader_comment)

for i in range(len(comment_lines)):
  line = comment_lines[i]
  d = model.test_results[i]
  hyp = d['hyp'].item()
  prob = d['prob'].item() 
  # if hyp == 1 and prob > 0.9: # 1である確率が0.9以上のものに厳選
  print(f"{i+1}\t{hyp}\t{prob}\t{line}")

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'accuracy': 0.43968871235847473}
--------------------------------------------------------------------------------
1	1	0.5719050765037537	0
2	1	0.8032328486442566	"DAT GUM IT ITS TRUMP JUST BEING TRUMP Y&quot;ALL CITY SLICKERS DONT GIT IT NOW BACK TO HUNTER BIDENS LAP TOP  ,,,,CANT <br>FIND A DAMMM THING HOWEVER FOLLOWING FOX NEWS TALKING POINTS ⤵️<br>,# 1 TO LIE AND OR DEFLECT <br># 2  RIGHT IS RIGHT EVEN WHEN WRONG ,( RIGHT WING ) THAT IS !!<br># 3 TRUMP IS GOD OUR LORD AND SAVIOR !!!!!<br># 4 GUNS GUNS AND GUNS DONT YOU LOVE THEM ,,,MAKE SURE PRE SCHOOL KIDS LEARN TO CLEAN THEIR RIFLES <br>( FOX NEWS TALKING POINTS)"
3	0	0.9797759056091309	This committee is designed for possible legislation. CNN is using this to turn people.<br><br>This committee will recommend prosecution but nothing will happen. <br><br>We know Trump did nothing to stop it
4	1	0.5751882195472717	Good let&#39;